## Algoritmos Genéticos 

In [1]:
import numpy as np


OBJECT_ARRAY = (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15)
WEIGHT_ARRAY = (63, 21, 2, 32, 13, 80, 19, 37, 56, 41, 14, 8, 32, 42, 7)
VALUE_ARRAY = (13, 2, 20, 10, 7, 14, 7, 2, 2, 4, 16, 17, 17, 3, 21)
WEIGHT_LIMIT = 275


class Backpack(object):
    def __init__(self, objects=dict()):
        self.objects = objects
        self.weight = sum(obj['weight'] for obj in objects.values())
        self.value = sum(obj['value'] for obj in objects.values())
    
    def add_obj_from_representation(self, representation, objects):
        for index, char in enumerate(representation):
            if char == '1':
                obj = objects[index]
                self.add(obj)
    
    def mutate(self, index, objects):
        if index in self.objects.keys():
            self.remove(index)
        
        else:
            obj = objects[index - 1]
            self.add(obj)

    def add(self, obj):
        self.objects[obj['id']] = obj

        self.weight += obj['weight']
        self.value += obj['value']

    def remove(self, obj_id):
        self.weight -= self.objects[obj_id]['weight']
        self.value -= self.objects[obj_id]['value']

        self.objects.pop(obj_id, None)

    def representation(self):
        # Binary representation:
        # 0 = not in backpack
        # 1 = in backpack
        representation = np.zeros(len(objects), dtype=int)

        for obj in self.objects:
            representation[obj - 1] = 1

        # Return representation as string
        return "".join(str(num) for num in representation)


objects = list()

for id, weight, value in zip(OBJECT_ARRAY, WEIGHT_ARRAY, VALUE_ARRAY):
    new_object = {
        'id': id, 
        'weight': weight, 
        'value': value
    }
    objects.append(new_object)

len(objects)

15

In [2]:
from random import sample

def generate_random_solution(objects):
    shuffle_objects = sample(objects, len(objects))
    solution = Backpack(dict())

    for obj in shuffle_objects:
        if solution.weight + obj['weight'] <= WEIGHT_LIMIT:
            solution.add(obj)
    
    return solution

In [3]:
from math import floor
from random import choice, random, randint
from copy import deepcopy

# Genetic algorithm
N = 50  # Number of generations before stopping
P = 20  # Number of solutions in initial population
REPRODUCTION_RATE = 1
MUTATION_RATE = 0.05

# Get best chromosome based on value
most_fit = lambda generation: max(generation,
                                  key=lambda solution: solution.value)

# Generate initial population
initial_population = list()

for _ in range(P):
    chromosome = generate_random_solution(objects)
    initial_population.append(chromosome)

best_solution = most_fit(initial_population)
generation = np.array(initial_population.copy())

for _ in range(N):
    # Randomly select based on fitness
    total_value = sum([solution.value for solution in generation])
    probabilities = [solution.value / total_value for solution in generation]

    reproductions = floor(P * REPRODUCTION_RATE)

    chromosomes = np.random.choice(generation, size=reproductions, 
                                    replace=False, p=probabilities)

    # For every 2 chromosomes
    for chromosome1, chromosome2 in zip(chromosomes[0::2], chromosomes[1::2]):
        # Randomly select crossing point
        crossing_point = choice(range(1, 15))

        representation1 = chromosome1.representation()[:crossing_point]
        representation2 = chromosome1.representation()[crossing_point:]

        # Cross chromosomes
        new_representation = representation1 + representation2
        new_chromosome = Backpack(dict())
        new_chromosome.add_obj_from_representation(new_representation, objects)

        # Add new chromosome to population
        np.append(generation, new_chromosome)

    # Mutate generation
    for chromosome in generation:
        mutation = random() <= MUTATION_RATE

        if mutation is True:
            mutation_index = randint(1, 15)
            chromosome.mutate(mutation_index, objects)

    possible_solutions = [chromosome for chromosome in generation
                          if chromosome.weight <= WEIGHT_LIMIT]
    best_generation_solution = most_fit(possible_solutions)

    if best_generation_solution.value > best_solution.value:
        best_solution = deepcopy(best_generation_solution)

    # Adjust population size
    total_value = sum([chromosome.value for chromosome in generation])
    probabilities = [chromosome.value / total_value for chromosome in generation]

    generation = np.random.choice(generation, size=P, 
                                  replace=False, p=probabilities)

best_solution.representation(), best_solution.value, best_solution.weight

('101011110011101', 134, 275)